1. Python을 통한 DB Connection

## 1-1 Postgresql 라이브러리(psycopg2)를 활용한 Connection

#### 1) Connection

In [71]:
import psycopg2

# database connection 생성
db = psycopg2.connect(
    host='127.0.0.1',
    dbname='postgres',
    user='postgres',
    password='',
    port=5433
    )

# 커서 생성 > 특정 SQL 문장을 처리한 결과를 담고 있는 영역을 가리키는 일종의 포인터 / 쿼리문에 의해서 반환되는 결과값들을 저장하는 메모리공간
cursor=db.cursor()

### 2) CRUD Execution

In [74]:
create_query = "CREATE TABLE lecture (id SERIAL PRIMARY KEY, name VARCHAR(65533), year INT, gender VARCHAR(65533), count INT);"

insert_query = "INSERT INTO lecture VALUES(1, 'Tom', 2024, 'M', 1100);"

update_query =  """
    UPDATE lecture
    SET id = 1,
        name = 'CHUNJAE',
        year = '1994',
        gender = 'M',
        count = '1000'
    WHERE id = 1;
    """

delete_query = "DELETE FROM lecture WHERE id = 1;"

drop_query = "DROP TABLE lecture"


In [75]:
# Create table

cursor.execute(create_query)
db.commit()

In [11]:
# INSERT ROW

cursor.execute(insert_query)
db.commit()

In [20]:
# SELECT

cursor.execute("SELECT * FROM lecture")
cursor.fetchall()

UndefinedTable: 오류:  "lecture" 이름의 릴레이션(relation)이 없습니다
LINE 1: SELECT * FROM lecture
                      ^


In [17]:
#Update ROW

cursor.execute(update_query)
db.commit()

cursor.execute("SELECT * FROM lecture")
cursor.fetchall()

[(1, 'CHUNJAE', 1994, 'M', 1000)]

In [70]:
# DELETE ROW

cursor.execute(delete_query)
db.commit()

cursor.execute("SELECT * FROM lecture")
cursor.fetchall()

InFailedSqlTransaction: 오류:  현재 트랜잭션은 중지되어 있습니다. 이 트랜잭션을 종료하기 전까지는 모든 명령이 무시될 것입니다


In [80]:
# DROP TABLE

cursor.execute(drop_query)
db.commit()

#### 3) Connection Pool

<img src="https://velog.velcdn.com/images/newnew_daddy/post/f0569aa7-1aad-466e-a24f-5e3b5f248a72/image.png" width="30%">

- 일정량의 Connection 객체를 미리 만들어서 pool에 저장
- 클라이언트 요청이 오면 Connection 객체를 빌려주고 해당 객체의 임무가 완료되면 다시 Connection 객체를 반납 받아 pool에 저장
- 큰 커넥션 풀은 메모리 소모가 큰 대신 대기 시간이 적어지고, 작은 커넥션 풀은 메모리 소모가 작은 대신 대기 시간이 길어진다. ([적정 Connection 수 공식](https://github.com/brettwooldridge/HikariCP/wiki/About-Pool-Sizing#the-formula))
- 자원을 사용하면 반드시 반납을 해줘야하는데 이를 위해 DB 연결시 파이썬 `with문`을 사용

In [81]:
# Connection 종료

cursor.close()

### 1-2. ORM 라이브러리(sqlalchemy)를 활용한 Connection

#### ORM(Object Relational Mapping)

<img src="https://velog.velcdn.com/images/newnew_daddy/post/ca5652a2-0686-462b-bed4-4483815708e5/image.png" width="40%">

- Python 객체와 관계형 DB의 data를 매핑해주는 것
- 장점
    - Query가 아닌 코드로 데이터를 조작할 수 있어 개발자가 프로그래밍에 더 집중할 수 있도록 도와준다.
    - 재사용 및 유지보수의 편리성이 증가한다.
    - DBMS에 대한 종속성이 줄어든다.

- 단점
  - 완벽한 ORM으로만 서비스를 구현하기 어렵다.
  - 프로시저가 많은 시스템에선 ORM의 객체 지향적인 장점을 활용하기 어렵다.

#### 1) Connection 및 Session

In [82]:
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker

engine_name = 'postgresql'
user_id = 'postgres'
user_pw = ''
host = '127.0.0.1'
port = '5433'
database = 'postgres'

db = create_engine(f'{engine_name}://{user_id}:{user_pw}@{host}:{port}/{database}')

cursor = db.connect()
Session = sessionmaker(db)
session = Session

In [62]:
cursor.close()

In [83]:
from sqlalchemy import Column, INTEGER, VARCHAR
from sqlalchemy.orm import declarative_base

Base = declarative_base()

class lecture(Base):
    __tablename__ = 'lecture'

    id = Column('id', INTEGER, primary_key=True, autoincrement=True)
    name = Column('name', VARCHAR(65533), nullable=False)
    year = Column('year', INTEGER, nullable=False)
    gender = Column('gender',VARCHAR(65533), nullable=False)
    count = Column('count', INTEGER, nullable=False)

# Create
Base.metadata.create_all(db)

### 3) ORM을 활용한 CRUD

In [84]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=db)
session = Session()


res = session.query(lecture).first()

print(res.id, res.name, res.year, res.gender, res.count)

AttributeError: 'NoneType' object has no attribute 'id'

In [85]:
session = Session()


res = session.query(lecture).first()

print(res.id, res.name, res.year, res.gender, res.count)

AttributeError: 'NoneType' object has no attribute 'id'

In [86]:
# INSERT

datal = lecture(id=3, name='chunjae', year=1990, gender='M', count=1234)
session.add(datal)
session.commit()

In [40]:
# UPDATE

session.query(lecture).filter(lecture.name == 'chunjae').update({'gender':'F', 'count':1500})
session.commit()

In [87]:
# DELETE

session.query(lecture).filter(lecture.id == 3).delete()
session.commit()

#### 4) OMR Core 방법을 활용한 CRUD

In [92]:
from sqlalchemy import create_engine, Table, Column, Integer, String, MetaData
from sqlalchemy.orm import sessionmaker

db = create_engine(f'{engine_name}://{user_id}:{user_pw}@{host}:{port}/{database}')

Session = sessionmaker(db)
session = Session()

meta = MetaData()

core_table = Table(
    'first', meta,
    Column('id', Integer, primary_key=True, autoincrement=True),
    Column('name', String),
    Column('year', Integer),
    Column('gender', String),
    Column('count', Integer)
)

meta.create_all(db)

In [101]:
from sqlalchemy import insert, update, delete

# Select ALL
res = core_table.select()
result = session.execute(res)

for row in result:
    print(row)

(1, 'hahaha', 2023, 'M', 555)
(2, 'hwang', 1994, 'M', 12)


In [94]:
# Insert one
stmt = insert(core_table).values(name="hahaha", year=2023, gender="M", count=555)

with db.connect() as conn:
    result = conn.execute(stmt)
    conn.commit()

In [96]:
# Insert multi
stmt = insert(core_table)
data_list = [
    {"name": "hwang", "year": 1994, "gender": "M", "count": 12},
    {"name": "jeong", "year": 1996, "gender": "F", "count": 32}
]

with db.connect() as conn:
    result = conn.execute(stmt, data_list)
    conn.commit()

In [98]:
#Update 
stmt = update(core_table).where(core_table.c.name == "jeong").values(year=1000)

with db.connect() as conn:
    result = conn.execute(stmt)
    conn.commit()

In [100]:
# Delete
stmt = delete(core_table).where(core_table.c.name == "jeong")

with db.connect() as conn:
    result = conn.execute(stmt)
    conn.commit()

#### 1-3. Python을 활용한 Bulk Insert

##### 1.For문을 활용한 Insert
##### 2.to_sql() 함수를 활용한 Insert
##### 3.execute_values() 함수를 활용한 Insert

#### Panda for loop

In [5]:
import psycopg2 as pgsql

# database connection
db = pgsql.connect(
    host='127.0.0.1',
    dbname='postgres',
    user='postgres',
    password='',
    port=5433
)

cursor = db.cursor()

In [6]:
drop_query = "DROP TABLE lecture"

cursor.execute(drop_query)
db.commit()

In [7]:
create_query = "CREATE TABLE lecture (id SERIAL PRIMARY KEY, name VARCHAR(65533), year INT, gender VARCHAR(65533), count INT);"

cursor.execute(create_query)
db.commit()

In [10]:
import pandas as pd

# Import
df = pd.read_csv('./dataset/data-01/names.csv')
df.head(10)

,id,name,year,gender,count
0,1,Mary,1880,F,7065
1,2,Anna,1880,F,2604
2,3,Emma,1880,F,2003
3,4,Elizabeth,1880,F,1939
4,5,Minnie,1880,F,1746
5,6,Margaret,1880,F,1578
6,7,Ida,1880,F,1472
7,8,Alice,1880,F,1414
8,9,Bertha,1880,F,1320
9,10,Sarah,1880,F,1288


In [11]:
df.dtypes

id         int64
name      object
year       int64
gender    object
count      int64
dtype: object

In [22]:
delete_query = "DELETE FROM lecture"
cursor.execute(delete_query)

db.commit()

In [20]:
# BULK INSERT -> tuple(for)

for data in range(len(df)):
    datas = tuple(df.iloc[data])
    query = f"INSERT INTO lecture VALUES{datas}"
    cursor.execute(query)

db.commit()

In [23]:
# SELECT
cursor.execute("SELECT * FROM lecture LIMIT 10")
cursor.fetchall()

[]

#### 2) Pandas - to_sql() 메소드

- dataframe.to_sql(테이블명, sqlalchemy_connector, option)

In [8]:
from sqlalchemy import create_engine

engine = 'postgresql'
user = 'postgres'
password = ''
host = '127.0.0.1'
port = '5432'
database = 'postgres'

db = create_engine(f'{engine}://{user}:{password}@{host}:{port}/{database}')

In [9]:
import pandas as pd

df = pd.read_csv('./dataset/data-01/names.csv')
df.iloc(1)
df.iloc[1,:]

id           2
name      Anna
year      1880
gender       F
count     2604
Name: 1, dtype: object

In [10]:
# BULK INSERT
df.to_sql(name="lecture", con=db, if_exists="replace")

1000

### 3) psycopg2 라이브러리의 execute_values() 메소드

In [2]:
import psycopg2

db = psycopg2.connect(
    host='127.0.0.1',
    dbname='postgres',
    user='postgres',
    password='',
    port=5433
)

cursor=db.cursor()

In [3]:
delete_query = "DELETE fROM lecture;"

cursor.execute(delete_query)
db.commit()

In [4]:
cursor.execute("SELECT * FROM lecture")
cursor.fetchall()

[]

In [5]:
import pandas as pd

df = pd.read_csv('./dataset/data-01/names.csv')
df['id'] = df['id'].astype(float)
df['year'] = df['year'].astype(float)
df['count'] =  df['count'].astype(float)

insert_list = list()

for i in range(5,50):
    insert_list.append(tuple(df.iloc[i]))

In [7]:
df.dtypes

id        float64
name       object
year      float64
gender     object
count     float64
dtype: object

In [8]:
import numpy as np
from psycopg2.extensions import register_adapter, AsIs
register_adapter(np.int64, AsIs)

In [9]:
from psycopg2.extras import execute_values

sql = f"INSERT INTO lecture VALUES %s;"
execute_values(cursor, sql, insert_list)

db.commit()

### 2. dotenv 라이브러리를 활용한 민감 정보 관리

- 환경 변수에 대한 관리를 효과적이고 안전하게 할 수 있도록 도와주는 python 라이브러리
- DB정보, 비밀번호, API KEY 등 외부에 공유되거나 Git 에 올라가면 안되는 값들을 하드코딩 하지 않고 사용

pip install python-dotenv

In [11]:
import dotenv

# PATH
env_path = dotenv.find_dotenv()
print(env_path)

c:\Users\user\Desktop\code\.env


In [13]:
# .env load
dotenv.load_dotenv(env_path)

True

In [ ]:
# .env values
dotenv.dotenv_values(env_path)

In [15]:
# .env values change
# dotenv.set_key(env_path, 'HOST', '876543')

(True, 'HOST', '876543')

In [30]:
# LOAD

import os
from dotenv import load_dotenv

load_dotenv()

engine   = os.getenv("POSTGRES_ENGINE")
user     = os.getenv("POSTGRES_USER")
password = os.getenv("POSTGRES_PASSWORD")
host     = os.getenv("POSTGRES_HOST")
port     = os.getenv("POSTGRES_PORT")
database = os.getenv("POSTGRES_DB")

In [ ]:
(engine, user, password, host, port, database)

### DB Connection

In [33]:
import psycopg2

db = psycopg2.connect(
    dbname=database,
    user=user,
    password=password,
    port=port,
    host=host
)

cursor=db.cursor()

### 3. Python Class를 사용한 DB Connection

##### 0) self

In [34]:
class TempClass:
    def __init__(self, name:str, age:int, count:int):
        self.name = name
        self.age = age
        self.count = count
        self.addage()

    def printall(self):
        print(self.name)
        print(self.age)
        print(self.count)

    def printself(self):
        return self
    
    def addage(self):
        self.new_age = self.age+10

In [36]:
a1 = TempClass(age=2,count=3,name="chunjae")
s1 = a1.printself()

print(a1)
print(s1)

In [37]:
# 객체 인자 확인
s1.__dict__

{'name': 'chunjae', 'age': 2, 'count': 3, 'new_age': 12}

In [38]:
[i for i in dir(s1) if '__' not in i]

['addage', 'age', 'count', 'name', 'new_age', 'printall', 'printself']

In [41]:
from easydict import EasyDict

self = EasyDict(dict())
self["name"] = "jeongseok"
self["age"] = 3
self["count"] = 1

s2 = self

s2

{'name': 'jeongseok', 'age': 3, 'count': 1}

In [42]:
self.hoho = "ho"

self

{'name': 'jeongseok', 'age': 3, 'count': 1, 'hoho': 'ho'}

### 1) DBConnector Class

In [43]:
import psycopg2

class DBConnector:
    def __init__(self, host, database, user, password, port):
        self.host = host
        self.database = database
        self.user = user
        self.password = password
        self.port = port

    def postgres_connect(self):
        print('<<< pssql_connector >>')
        self.conn = psycopg2.connect(
                                        host=self.host, \
                                        dbname=self.database, \
                                        user=self.user, \
                                        password=self.password, \
                                        port=self.port
                                    )
        return self

In [ ]:
(host, database, user, password, port)

In [45]:
db_object = DBConnector(host, database, user, password, port)
db_conn = db_object.postgres_connect().conn

cursor=db_conn.cursor()

<<< pssql_connector >>


In [46]:
cursor.execute("SELECT * FROM lecture LIMIT 5")
cursor.fetchall()

[(6, 'Margaret', 1880, 'F', 1578),
 (7, 'Ida', 1880, 'F', 1472),
 (8, 'Alice', 1880, 'F', 1414),
 (9, 'Bertha', 1880, 'F', 1320),
 (10, 'Sarah', 1880, 'F', 1288)]

##### 2) with문 적용
Bad Case

<img src="https://velog.velcdn.com/images/newnew_daddy/post/df312bda-0b22-4476-8a03-505f1d3cf5b4/image.png" width="20%">

Good Case

<img src="https://velog.velcdn.com/images/newnew_daddy/post/707d46b0-8a0b-4862-ad95-285dc04ddc29/image.png" width="20%">

- Python에서 파일 또는 리소스 관리를 더 효과적으로 처리하기 위한 블록 구조
- 리소스를 열고 사용한 후 자동으로 닫아주기 때문에 닫아주는 코드를 작성할 필요가 없습니다. 
- 주로 파일 입출력, 데이터베이스 연결, 네트워크 연결 등 리소스 관리에 사용됩니다.

    ```
    with문을 사용할 때는 with 키워드 다음에 리소스를 관리하는 객체를 생성하는 표현식을 사용하며, 
    이 객체는 __enter__와 __exit__ 메소드를 구현해야 합니다. 
    with 블록 내에서 리소스를 사용하고 블록을 벗어나면 __exit__ 메소드가 호출되어 리소스를 정리합니다.

    -ChatGPT-
    ```

In [ ]:
file = open('temp.txt', 'a')
file.write("1")
file.write("2")

file.close()

In [48]:
db_object = DBConnector(host, database, user, password, port)

with db_object as connected:
    db_conn = connected.conn
    cursor=db_conn.cursor()

    cursor.execute("SELECT * FROM lecture LIMIT 5")
    print(cursor.fetchall())

TypeError: 'DBConnector' object does not support the context manager protocol

In [49]:
class ContextManager:
    def __init__(self, age):
        print("Start! Class")
        self.age = age

    def __enter__(self):
        print("Enter")

    def __exit__(self, exc_type, exc_value, traceback):
        print("Exit")

    def show(self):
        print(self.age)
        return self.__dict__

In [50]:
context = ContextManager(age=10)

with context:
    print(context.show())

Start! Class
Enter
10
{'age': 10}
Exit


In [54]:
class DBConnector:
    def __init__(self, host, database, user, password, port):
        print("Start! Class")
        self.host = host
        self.database = database
        self.user = user
        self.password = password
        self.port = port
        self.connect = self.postgres_connect()

    def __enter__(self):
        print("Enter")
        return self
    
    def __exit__(self, exc_type, exc_value, traceback):
        self.conn.close()
        print("Exit")

    def postgres_connect(self):
        self.conn = psycopg2.connect(
                                        host=self.host, \
                                        dbname=self.database, \
                                        user=self.user, \
                                        password=self.password, \
                                        port=self.port
                                    )
        return self

In [55]:
db_object = DBConnector(host, database, user, password, port)
db_conn = db_object.postgres_connect().conn

cursor=db_conn.cursor()

Start! Class


In [56]:
db_object = DBConnector(host, database, user, password, port)

with db_object as connected:
    db_conn = connected.conn
    cursor=db_conn.cursor()

    cursor.execute("SELECT * FROM lecture LIMIT 10")
    print(cursor.fetchall())

Start! Class
Enter
[(6, 'Margaret', 1880, 'F', 1578), (7, 'Ida', 1880, 'F', 1472), (8, 'Alice', 1880, 'F', 1414), (9, 'Bertha', 1880, 'F', 1320), (10, 'Sarah', 1880, 'F', 1288), (11, 'Annie', 1880, 'F', 1258), (12, 'Clara', 1880, 'F', 1226), (13, 'Ella', 1880, 'F', 1156), (14, 'Florence', 1880, 'F', 1063), (15, 'Cora', 1880, 'F', 1045)]
Exit


In [57]:
class DBConnector:
    def __init__(self, host, database, user, password, port):
        print("Start! Class")
        self.conn_params = dict(
            host = host,
            dbname = database,
            user = user,
            password = password,
            port = port
        )
        self.host = host
        self.database = database
        self.user = user
        self.password = password
        self.port = port

    def __enter__(self):
        print("Enter")
        return self
    
    def __exit__(self, exc_type, exc_value, traceback):
        self.conn.close()
        print("Exit")

    def postgres_connect(self):
        print("<<< postgres_connect >>>")
        self.conn = psycopg2.connect(
                                        host=self.host, \
                                        dbname=self.database, \
                                        user=self.user, \
                                        password=self.password, \
                                        port=self.port
                                    )
        return self

#### Asterisk(*)

In [58]:
def func(one, two):
    print("one = ", one)
    print("two = ", two)

func(1,2)

one =  1
two =  2


In [59]:
# *args -> 이름이 정해지지 않은 변수가 여러개 있을때 사용

def func(*args):
    print("*args = ", args)
    print(sum(args))

func(1,2,3)

*args =  (1, 2, 3)
6


In [63]:
# **kwargs -> 이름이 정해진 변수가 여러개 있을때 사용

def func(**kwargs):
    print("**kwargs = ", kwargs)

func(one=1,two=2)

**kwargs =  {'one': 1, 'two': 2}


In [69]:
test = DBConnector(1,2,3,4,5)

test.conn_params

Start! Class


{'host': 1, 'dbname': 2, 'user': 3, 'password': 4, 'port': 5}

In [70]:
# LIST
args = [1,2,3,4,5]

list_params = DBConnector(*args)

list_params.conn_params

Start! Class


{'host': 1, 'dbname': 2, 'user': 3, 'password': 4, 'port': 5}

In [71]:
# DICT

kwargs = dict(
            host = 1,
            database = 2,
            user = 3,
            password = 4,
            port =5
)

dict_params = DBConnector(**kwargs)

dict_params.conn_params

Start! Class


{'host': 1, 'dbname': 2, 'user': 3, 'password': 4, 'port': 5}

In [5]:
class DBConnector:
    def __init__(self, host, database, user, password, port):
        print("Start DBConnector!")
        self.conn_params = dict(
            host = host,
            dbname = database,
            user = user,
            password = password,
            port = port
        )
        self.connect = self.postgres_connect()

    def __enter__(self):
        print("Enter")
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        self.conn.close()
        print("Exit")

    def postgres_connect(self):
        self.conn = psycopg2.connect(**self.conn_params)
        return self

In [75]:
db_object = DBConnector(host, database, user, password, port)

with db_object as connected:
    db_conn = connected.conn
    cursor=db_conn.cursor()

    cursor.execute("SELECT * FROM lecture LIMIT 5")
    print(cursor.fetchall())

Start DBConnector!
Enter
[(6, 'Margaret', 1880, 'F', 1578), (7, 'Ida', 1880, 'F', 1472), (8, 'Alice', 1880, 'F', 1414), (9, 'Bertha', 1880, 'F', 1320), (10, 'Sarah', 1880, 'F', 1288)]
Exit


#### .env 적용한 connection

In [1]:
import dotenv, psycopg2

env_path = dotenv.find_dotenv()
# print(env_path)
dotenv.load_dotenv(env_path)

import os
from dotenv import load_dotenv

load_dotenv()

engine   = os.getenv("POSTGRES_ENGINE")
user     = os.getenv("POSTGRES_USER")
password = os.getenv("POSTGRES_PASSWORD")
host     = os.getenv("POSTGRES_HOST")
port     = os.getenv("POSTGRES_PORT")
database = os.getenv("POSTGRES_DB")

In [2]:
DB_SETTINGS = {
    "POSTGRES": {
        'host': os.getenv("POSTGRES_HOST"),
        'database': os.getenv("POSTGRES_DB"),
        'user': os.getenv("POSTGRES_USER"),
        'password': os.getenv("POSTGRES_PASSWORD"),
        'port': os.getenv("POSTGRES_PORT")
    }
}

In [ ]:
DB_SETTINGS['POSTGRES']

In [6]:
db_object = DBConnector(**DB_SETTINGS['POSTGRES'])

with db_object as connected:
    db_conn = connected.conn
    cursor=db_conn.cursor()

    cursor.execute("SELECT * FROM lecture LIMIT 5")
    print(cursor.fetchall())

Start DBConnector!
Enter
Exit


UndefinedTable: 오류:  "lecture" 이름의 릴레이션(relation)이 없습니다
LINE 1: SELECT * FROM lecture LIMIT 5
                      ^


In [83]:
DB_SETTINGS = {
    "POSTGRES": {
        'host': os.getenv("POSTGRES_HOST"),
        'database': os.getenv("POSTGRES_DB"),
        'user': os.getenv("POSTGRES_USER"),
        'password': os.getenv("POSTGRES_PASSWORD"),
        'port': os.getenv("POSTGRES_PORT")
    },
    "KDT9": {
        'host': os.getenv("POSTGRES_HOST"),
        'database': os.getenv("POSTGRES_DB_2"),
        'user': os.getenv("POSTGRES_USER"),
        'password': os.getenv("POSTGRES_PASSWORD"),
        'port': os.getenv("POSTGRES_PORT")
    }
}

In [85]:
db_object = DBConnector(**DB_SETTINGS['POSTGRES'])

with db_object as connected:
    db_conn = connected.conn
    cursor=db_conn.cursor()

    cursor.execute("SELECT * FROM INFORMATION_SCHEMA.TABLES")
    print(cursor.fetchall())

Start DBConnector!
Enter
[('postgres', 'pg_catalog', 'pg_statistic', 'BASE TABLE', None, None, None, None, None, 'YES', 'NO', None), ('postgres', 'pg_catalog', 'pg_type', 'BASE TABLE', None, None, None, None, None, 'YES', 'NO', None), ('postgres', 'pg_catalog', 'pg_foreign_table', 'BASE TABLE', None, None, None, None, None, 'YES', 'NO', None), ('postgres', 'pg_catalog', 'pg_authid', 'BASE TABLE', None, None, None, None, None, 'YES', 'NO', None), ('postgres', 'pg_catalog', 'pg_shadow', 'VIEW', None, None, None, None, None, 'NO', 'NO', None), ('postgres', 'public', 'first', 'BASE TABLE', None, None, None, None, None, 'YES', 'NO', None), ('postgres', 'pg_catalog', 'pg_roles', 'VIEW', None, None, None, None, None, 'NO', 'NO', None), ('postgres', 'pg_catalog', 'pg_statistic_ext_data', 'BASE TABLE', None, None, None, None, None, 'YES', 'NO', None), ('postgres', 'pg_catalog', 'pg_hba_file_rules', 'VIEW', None, None, None, None, None, 'NO', 'NO', None), ('postgres', 'pg_catalog', 'pg_settings'